# Update Countries

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import ipywidgets as widgets
from ipywidgets import HBox, VBox, HTML
from IPython.display import display
from giga.app.config import get_registered_countries
from giga.viz.notebooks.components.html.sections import section

option = widgets.RadioButtons(
    options=['Update an existing country', 'Add a new country'],
    disabled=False,
    index=None
)

country_select = widgets.Dropdown(
    options=get_registered_countries(),
    description='Country:',
)

country_input = widgets.Text(
    description='Country:',
)

output = widgets.Output()
country_config_output = widgets.Output()
optional_info_output = widgets.Output()
save_output = widgets.Output()
full_config_output = widgets.Output()

first_time = True
def on_change(change):
    global first_time
    output.clear_output()
    with output:
        if change['new'] == 'Update an existing country':
            display(country_select)
        else:
            display(country_input)
    if not first_time:
        return
    first_time = False
    with full_config_output:
        display(country_config_output)
        display(optional_info_output)
        display(save_output)

# Set the function to be called when the selected option changes
option.observe(on_change, 'value')

# Display the option
display(section(
    title="Select Country",
    contents=VBox([
        option,
        output
    ]),
    extra_class="header"))
display(full_config_output)

def creating_new_country():
    return option.value == 'Add a new country'

def get_selected_country():
    if not creating_new_country():
        if country_select.value is not None:
            return country_select.value
    elif country_input.value.strip() != "":
        return country_input.value.strip().replace(" ", "_").lower()
    return None

In [ ]:
import json


from giga.utils.globals import COUNTRY_DEFAULT_RELATIVE_DIR
from giga.data.space.country_updater import CountryUpdateRequest
from giga.viz.notebooks.components.widgets.giga_file_upload import GigaFileUpload
from giga.viz.notebooks.components.html.sections import section
from giga.viz.notebooks.cost_estimation_parameter_input import CostEstimationParameterInput


giga_country_code = widgets.Text()

country_center_lat = widgets.FloatText()
country_center_long = widgets.FloatText()

def up_btn(description, accept=".csv", **kwargs):
    layout = widgets.Layout(width='300px')
    return widgets.FileUpload(accept=accept, multiple=False, 
                              description=description, layout=layout, 
                              max_file_size=100,  # MB
                              **kwargs)

# Setting up country defaults

defaults_upload_output = widgets.Output()
upload_defaults = GigaFileUpload(accept='.json', multiple=True, description="Import Config")
default_params = CostEstimationParameterInput(
    local_data_workspace='workspace',
    show_map=False,
    school_selection=None)
def on_upload_change(change):
    try:
        imported = json.loads(upload_defaults.value[0].content.tobytes())
    except IndexError:
        imported = {}
    defaults_upload_output.clear_output()
    default_params.update(imported)
upload_defaults.observe(on_upload_change, names='value')

country_defaults = up_btn(accept='.csv', description="Country defaults (.csv)")
# display(country_defaults)


with country_config_output:
    display(section(
        title="Country Configuration",
        contents=VBox([
            HBox([
                HTML("Giga country code:"),
                giga_country_code
            ]),
            HBox([
                HTML("Country center:"),
                VBox([
                    HBox([HTML("Lat:"), country_center_lat]),
                    HBox([HTML("Long:"), country_center_long])
                ])
            ]),
            upload_defaults,
            default_params.country_default_parameter_input()
        ])
    ))

In [ ]:
cellular = up_btn(accept='.csv', description="Cell tower locations (.csv)")
fiber = up_btn(accept='.csv', description="Fiber node locations (.csv)")
schools_supplemental = up_btn(accept='.csv', description="Supplemental school info (.csv)")

upload_output = widgets.Output()
upload_button = widgets.Button(description="Upload")

with optional_info_output:
    display(section(
        title="Optional information",
        contents=VBox([
            cellular,
            fiber,
            schools_supplemental,
        ]),
        extra_class="center"
    ))

with save_output:
    display(section(
        title="Save",
        contents=VBox([
            upload_button,
            upload_output
        ]),
        extra_class="footer"
    ))

def on_upload_change(change):
    upload_output.clear_output()
    with upload_output:
        req = CountryUpdateRequest()
        req.country_name = get_selected_country()
        req.country_defaults = country_defaults
        req.cellular = cellular
        req.fiber = fiber
        req.schools_supplemental = schools_supplemental
        req.attempt()
        

upload_button.on_click(on_upload_change)